## Import libraries

In [1]:
import pandas as pd
import glob
import os
from tqdm import tqdm
import numpy as np
from datetime import timedelta

## Preprocess data functions

In [2]:
def getAllFiles(path, file_name_type, start_year, start_month, end_year, end_month, included_columns):
    li = []

    for x in tqdm(list(os.walk(path))):

        all_files = glob.glob(x[0] + "/*" + file_name_type + ".csv")

        for file_path in all_files:
            if (inTimePeriod(file_path, start_year, start_month, end_year, end_month)):
                df = pd.read_csv(file_path, index_col=None, header=0, usecols=included_columns)
                df = df[df['LSOA name'].notna()]
                df['LSOA'] = df['LSOA name'].apply(LSOANameToCode)
                df['MSOA'] = df['LSOA'].apply(LSOAtoMSOA)
                df = df.drop(columns=['LSOA name'])
                li.append(df)
                
    complete_df = pd.concat(li, axis=0, ignore_index=True)
    complete_df.rename_axis('index')

    return complete_df

def inTimePeriod(file_path, start_year, start_month, end_year, end_month):
    file_path_parts = file_path.split("\\")
    file_name = file_path_parts[len(file_path_parts) - 1]
    year = int(file_name[:4])
    month = int(file_name[5:7])
    start_bool = (year >= start_year) or (year == start_year and month >= start_month)
    end_bool = (year <= end_year) or (year == end_year and month <= end_month)
    if (start_bool and end_bool):
        return True
    return False

def monthYearToYear(monthYear):
    return int(monthYear[:4])

def monthYearToMonth(monthYear):
    return int(monthYear[5:7])

def MonthYearRegionDF(dataframe, month, year, region):
    return dataframe[(dataframe['Month'] == month) & (dataframe['Year'] == year) &
                     (dataframe['Falls within'] == region)]

def LSOANameToCode(LSOAname):
    return str(LSOAname)[-4:]

def LSOAtoMSOA(LSOA):
    return LSOA[:-1]

def countMSOA(df):
    df = df.groupby(['Month', 'MSOA', 'Crime type'])
    df = df.agg(count=('LSOA', 'count')).unstack(fill_value=0).stack()
    df = df.reset_index()
    return df

## Data location

In [ ]:
data_path = input("Enter the path to the data folder: ")

## Import data

Train set (2012-2018)

In [4]:
trainSet = getAllFiles(data_path, "street", 2012, 1, 2018, 12,
                       included_columns=['Month', 'LSOA name', 'Crime type'])
trainSet

100%|██████████| 132/132 [01:45<00:00,  1.25it/s]


,Month,Crime type,LSOA,MSOA
0,2012-01,Anti-social behaviour,001A,001
1,2012-01,Anti-social behaviour,001A,001
2,2012-01,Anti-social behaviour,001A,001
3,2012-01,Other theft,001A,001
4,2012-01,Other theft,001A,001
...,...,...,...,...
40834957,2018-12,Public order,062E,062
40834958,2018-12,Vehicle crime,062E,062
40834959,2018-12,Violence and sexual offences,062E,062
40834960,2018-12,Violence and sexual offences,062E,062


Test set no covid (split on March 2020)

In [5]:
# test set
testSetNoCovid = getAllFiles(data_path, "street", 2019, 1, 2020, 2,
                             included_columns=['Month', 'LSOA name', 'Crime type'])
testSetNoCovid

100%|██████████| 132/132 [00:32<00:00,  4.12it/s]


,Month,Crime type,LSOA,MSOA
0,2019-01,Anti-social behaviour,001A,001
1,2019-01,Criminal damage and arson,001A,001
2,2019-01,Criminal damage and arson,001A,001
3,2019-01,Other theft,001A,001
4,2019-01,Other theft,001A,001
...,...,...,...,...
12279031,2020-12,Criminal damage and arson,062E,062
12279032,2020-12,Violence and sexual offences,062E,062
12279033,2020-12,Violence and sexual offences,062E,062
12279034,2020-12,Violence and sexual offences,062E,062


Train set with no covid test data (2012-March 2020)

In [6]:
trainSetWithNoCovid = pd.concat([trainSet, testSetNoCovid], axis=0, ignore_index=True)
trainSetWithNoCovid

,Month,Crime type,LSOA,MSOA
0,2012-01,Anti-social behaviour,001A,001
1,2012-01,Anti-social behaviour,001A,001
2,2012-01,Anti-social behaviour,001A,001
3,2012-01,Other theft,001A,001
4,2012-01,Other theft,001A,001
...,...,...,...,...
53113993,2020-12,Criminal damage and arson,062E,062
53113994,2020-12,Violence and sexual offences,062E,062
53113995,2020-12,Violence and sexual offences,062E,062
53113996,2020-12,Violence and sexual offences,062E,062


Test set covid (split on March 2020)

In [7]:
# start of march covid measures in uk
testSetCovid = getAllFiles(data_path, "street", 2020, 3, 2021, 10,
                           included_columns=['Month', 'LSOA name', 'Crime type'])
testSetCovid

100%|██████████| 132/132 [00:29<00:00,  4.49it/s]


,Month,Crime type,LSOA,MSOA
0,2020-01,Anti-social behaviour,001A,001
1,2020-01,Burglary,001A,001
2,2020-01,Burglary,001A,001
3,2020-01,Other theft,001A,001
4,2020-01,Other theft,001A,001
...,...,...,...,...
10837197,2021-10,Criminal damage and arson,062E,062
10837198,2021-10,Public order,062E,062
10837199,2021-10,Vehicle crime,062E,062
10837200,2021-10,Violence and sexual offences,062E,062


## Replace values that changed throughout years

In [8]:
trainSet['Crime type'] = trainSet['Crime type'].replace('Violent crime', 'Violence and sexual offences')
trainSet['Crime type'] = trainSet['Crime type'].replace('Public disorder and weapons','Other crime')

In [9]:
testSetNoCovid['Crime type'] = testSetNoCovid['Crime type'].replace('Violent crime', 'Violence and sexual offences')
testSetNoCovid['Crime type'] = testSetNoCovid['Crime type'].replace('Public disorder and weapons','Other crime')

In [10]:
trainSetWithNoCovid['Crime type'] = trainSetWithNoCovid['Crime type'].replace('Violent crime', 'Violence and sexual offences')
trainSetWithNoCovid['Crime type'] = trainSetWithNoCovid['Crime type'].replace('Public disorder and weapons','Other crime')

In [11]:
testSetCovid['Crime type'] = testSetCovid['Crime type'].replace('Violent crime', 'Violence and sexual offences')
testSetCovid['Crime type'] = testSetCovid['Crime type'].replace('Public disorder and weapons','Other crime')

## Count crime type per MSOA per month

In [12]:
trainSetCount = countMSOA(trainSet)
trainSetCount

,Month,MSOA,Crime type,count
0,2012-01,001,Anti-social behaviour,6290
1,2012-01,001,Bicycle theft,0
2,2012-01,001,Burglary,1715
3,2012-01,001,Criminal damage and arson,1756
4,2012-01,001,Drugs,607
...,...,...,...,...
163459,2018-12,140,Robbery,4
163460,2018-12,140,Shoplifting,1
163461,2018-12,140,Theft from the person,2
163462,2018-12,140,Vehicle crime,12


In [13]:
testSetNoCovidCount = countMSOA(testSetNoCovid)
testSetNoCovidCount

,Month,MSOA,Crime type,count
0,2019-01,001,Anti-social behaviour,3402
1,2019-01,001,Bicycle theft,147
2,2019-01,001,Burglary,1473
3,2019-01,001,Criminal damage and arson,1699
4,2019-01,001,Drugs,397
...,...,...,...,...
46699,2020-12,140,Robbery,3
46700,2020-12,140,Shoplifting,1
46701,2020-12,140,Theft from the person,2
46702,2020-12,140,Vehicle crime,6


In [14]:
trainSetWithNoCovidCount = countMSOA(trainSetWithNoCovid)
trainSetWithNoCovidCount

,Month,MSOA,Crime type,count
0,2012-01,001,Anti-social behaviour,6290
1,2012-01,001,Bicycle theft,0
2,2012-01,001,Burglary,1715
3,2012-01,001,Criminal damage and arson,1756
4,2012-01,001,Drugs,607
...,...,...,...,...
210163,2020-12,140,Robbery,3
210164,2020-12,140,Shoplifting,1
210165,2020-12,140,Theft from the person,2
210166,2020-12,140,Vehicle crime,6


In [15]:
testSetCovidCount = countMSOA(testSetCovid)
testSetCovidCount

,Month,MSOA,Crime type,count
0,2020-01,001,Anti-social behaviour,3243
1,2020-01,001,Bicycle theft,167
2,2020-01,001,Burglary,1241
3,2020-01,001,Criminal damage and arson,1737
4,2020-01,001,Drugs,481
...,...,...,...,...
41155,2021-10,140,Robbery,0
41156,2021-10,140,Shoplifting,0
41157,2021-10,140,Theft from the person,0
41158,2021-10,140,Vehicle crime,0


## Change column names

In [16]:
# no covid
trainSetCount = trainSetCount.rename(columns={"Month":"Date"})
testSetNoCovidCount = testSetNoCovidCount.rename(columns={"Month":"Date"})
# covid
trainSetWithNoCovid = trainSetWithNoCovid.rename(columns={"Month":"Date"})
testSetCovidCount = testSetCovidCount.rename(columns={"Month":"Date"})

## Save as csv

In [17]:
trainSetCount.to_csv("no_covid_train.csv")

In [18]:
testSetNoCovidCount.to_csv("no_covid_test.csv")

In [19]:
trainSetWithNoCovidCount.to_csv("covid_train.csv")

In [20]:
testSetCovidCount.to_csv("covid_test.csv")